* Переименуем для удобства файл в hw_3_twitter.txt, так как по факту это не JSON, а текстовый файл, в котором каждая строка - JSON.
* Импортируем модуль JSON для преобразования строк в объекты.
* Затем циклом пройдёмся по всем строкам и преобразуем, скидывая объекты в список с именем twits.

In [2]:
%load_ext pycodestyle_magic
%pycodestyle_on

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [1]:
import json

f = open('hw_3_twitter.txt')
twits = []

for line in f:
    twit = json.loads(line)
    twits.append(twit)

#### 1. Сколько твитов в наборе?

Можем сразу посчитать количество твитов:

In [2]:
print(len(twits))

2556


#### 2. Какой процент твитов составляют удаленные записи? (помеченные как deleted)
Нетрудно заметить, что твиты, которые deleted, состоят из всего одной пары ключ-значение.

In [3]:
counter = 0
for twit in twits:
    if len(twit.items()) == 1:
        counter += 1
print(counter * 100 / len(twits))

14.162754303599375


#### 3. Какие самые популярные языки твитов?
Очевидно, что у каждого твита только один язык. Это облегчит нам задачу. Мы можем просто за один проход составить dict с парами язык-количество твитов, увеличивая количество твитов на единицу, как только нам попадается соответствующий язык.

In [4]:
import operator

count_of_langs = {}
for twit in twits:
    if twit.get('lang'):  # используем get() для проверки существования ключа
        lang = twit['lang']
        if count_of_langs.get(lang):
            count_of_langs[lang] += 1
        else:
            count_of_langs[lang] = 1

# Отсортируем же
sorted_count_of_langs = sorted(count_of_langs.items(),
                               key=operator.itemgetter(1), reverse=True)
top10 = sorted_count_of_langs[:10]
# Распечатаем топчик
for i in range(len(top10)):
    print(i+1, top10[i][0], top10[i][1])

1 en 719
2 ja 438
3 es 173
4 ko 149
5 th 123
6 ar 119
7 und 117
8 in 71
9 pt 69
10 fr 35


#### 4. Есть ли твиты от одного и того же пользователя? Если да, то сколько таких пользователей?
Подход строго аналогичный.

In [42]:
count_of_users = {}

for twit in twits:
    if twit.get('user'):
        user_id = twit['user']['id']
        if count_of_users.get(user_id):
            count_of_users[user_id] += 1
        else:
            count_of_users[user_id] = 1

count_of_users_gt_1 = 0
for key, value in count_of_users.items():
    if value > 1:
        count_of_users_gt_1 += 1

print(count_of_users_gt_1)

#### 5. Топ-20 хэштегов (для них есть специальное поле)
Почти то же самое, но с более глубоким погружением в структуру.

In [5]:
import operator

count_of_hashtags = {}

for twit in twits:
    if twit.get('entities'):  # энтити есть?
        entities = twit['entities']
        if entities.get('hashtags'):  # хэштеги есть?
            hashtags = entities['hashtags']
            if hashtags != []:  # а они не пустые?
                for hashtag in hashtags:
                    text = hashtag['text']  # выдернем текст
                    if count_of_hashtags.get(text):  # и тут как обычно
                        count_of_hashtags[text] += 1
                    else:
                        count_of_hashtags[text] = 1

sorted_count_of_hashtags = sorted(count_of_hashtags.items(),
                                  key=operator.itemgetter(1), reverse=True)
top20 = sorted_count_of_hashtags[:20]

for i in range(len(top20)):
    print(i+1, top20[i][0], top20[i][1])

1 BTS 17
2 방탄소년단 13
3 AMAs 11
4 人気投票ガチャ 8
5 태형 7
6 뷔 6
7 BTSinChicago 5
8 BTSLoveYourselfTour 5
9 오늘의방탄 5
10 PledgeForSwachhBharat 5
11 MPN 5
12 PCAs 4
13 V 4
14 시카고1회차공연 4
15 เป๊กผลิตโชค 4
16 JIMIN 4
17 running 3
18 NCT 3
19 지민 3
20 WajahmuPlastik 3


#### 6. Предобработать тексты оригинальных твитов (не ретвитов) на английском языке (убрать пунктуацию, привести к нижнему регистру) и составить частотный словарь

In [23]:
import operator
import string

punct_chars = string.punctuation

final_words = []
count_of_final_words = {}

for twit in twits:
    if ('en' == twit.get('lang')
       and not twit.get('retweeted_status')
       and not twit.get('quoted_status')):
        words = twit['text'].split()  # разобьём на слова
        for word in words:
            word = word.strip(punct_chars)
            word = word.lower()  # в нижний регистр
            if '' != word:
                final_words.append(word)


for word in final_words:
    if(count_of_final_words.get(word)):
        count_of_final_words[word] += 1
    else:
        count_of_final_words[word] = 1

sorted_count_of_final_words = sorted(count_of_final_words.items(),
                                     key=operator.itemgetter(1), reverse=True)
top10 = sorted_count_of_final_words[:10]

for i in range(len(top10)):
    print(i+1, top10[i][0], top10[i][1])

1 the 106
2 to 79
3 a 65
4 i 60
5 and 57
6 you 45
7 for 40
8 is 40
9 of 40
10 it 37


#### 7. Найти количество подписчиков (фолловеров) у авторов твитов и вывести топ-10
Почти то же, что и 5.

In [22]:
import operator

count_of_followers = {}

for twit in twits:
    if twit.get('user'):  # юзер есть?
        user = twit['user']
        if user.get('id') and user.get('followers_count'):  # фолловеры есть?
            count_of_followers[user['name']] = user['followers_count']

sorted_count_of_followers = sorted(count_of_followers.items(),
                                   key=operator.itemgetter(1), reverse=True)
top10 = sorted_count_of_followers[:10]

for i in range(len(top10)):
    print(i+1, top10[i][0], top10[i][1])

1 Filosofía♕ 2521403
2 FITNESS Magazine 1491309
3 malaysiakini.com 1206759
4 NYT Science 1137374
5 Gramática 625463
6 TGRT Haber 392472
7 The Sun Football ⚽ 383698
8 Melbourne, Australia 374222
9 Roznama Express 318189
10 💞 ცųཞɠɛཞცơơɠıɛ 💞 311319


#### 8.* Топ-10 источников твита (из какого приложения написан) (подсказка: для обработки используйте регулярные выражения)

Для работы с регэксами подключается модуль re.
Из поля source вот такого вида:

```html
<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>
```

нам надо вытащить то, что, грубо говоря, между > и <.
Строим регэкс вида:
```
"строка, начинающаяся с <a, потом много любых символов, потом >, потом ТО, ЧТО НАМ НУЖНО, до </a>, ну а тут и конец строки".
```

In [25]:
import re
import operator

count_of_sources = {}
sources = []

for twit in twits:
    if twit.get('source'):
        html_string = twit['source']
        regex = re.compile(r'^<a.*">(.*)<\/a>$')
        source = regex.findall(html_string)
        sources += source

for source in sources:
    if count_of_sources.get(source):  # и тут как обычно
        count_of_sources[source] += 1
    else:
        count_of_sources[source] = 1

sorted_count_of_sources = sorted(count_of_sources.items(),
                                 key=operator.itemgetter(1), reverse=True)
top10 = sorted_count_of_sources[:10]

for i in range(len(top10)):
    print(i+1, top10[i][0], top10[i][1])

1 Twitter for iPhone 800
2 Twitter for Android 695
3 Twitter Web Client 140
4 twittbot.net 122
5 Twitter Lite 51
6 Twitter for iPad 28
7 TweetDeck 23
8 Facebook 17
9 IFTTT 14
10 تطبيق قرآني 10
